# Use Random forest to drop the unimportant channels
* This notebook will use random forest to classify the data and sum the features importance obtains from random forest so that we know which channels are the most important and which channels are the least important
* The goal is to be able to using random forest as feature selection to select only the useful channels for classfication.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler
from mne.decoding import SPoC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as tts
import mne
from sklearn.model_selection import StratifiedKFold
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator,
                          cross_val_multiscore, LinearModel, get_coef)
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE 
import os
from mne.decoding import UnsupervisedSpatialFilter
import os.path as op
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from scipy.stats import skew, kurtosis
import mne 

from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import SVC  # noqa
from sklearn.model_selection import ShuffleSplit  # noqa

from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA
from scipy.stats import skew, kurtosis
from sklearn.metrics import confusion_matrix
from mne.viz import tight_layout

import time
import pandas as pd
from sklearn.ensemble import RandomForestRegressor


In [ ]:
%run Batch_ArtifactFilter_Epoch.ipynb

In [ ]:
epoch=remove_artifacts_and_create_epochs('D:\\USB Drive\\NewEEG-200s\\SA20140206\\20140206_1151.set')

In [ ]:
def removebadchannel(epoch):
    epoch.drop_channels(['Lm','Rm','VEOG'])
    epoch.drop_channels(['STI 014'])

In [ ]:
 removebadchannel(epoch)

In [ ]:
newdata=epoch.get_data()
epoch.get_data().shape

In [ ]:
## Methods for convert our events into binary, 4: Language, 5:Non Language
def filter_event_id(epoch):
    newepoch=epoch
    epochs=[[]]
    j=0
    for i in range(len(newepoch.events[:,-1])):
        if i % 3 ==0:
            epochs.append([])
            j=j+1
            epochs[j].append(epoch.events[i,-1])
        else:
            epochs[j].append(epoch.events[i,-1])
    for events in epochs:
        if  4 in events or 1 in events or 13 in events or 16 in events:
            for i in range(len(events)):
                    events[i]=4
        else:
            for i in range(len(events)):
                    events[i]=5
    epochs.pop(0)
    flattened_list = [y for x in epochs for y in x]
    for i in range(len(newepoch.events)):
        newepoch.events[i]=flattened_list[i]
    return newepoch

In [ ]:
epoch=filter_event_id(epoch)
print(epoch.events[:,-1])

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

In [ ]:
## Random Forest Classfier
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))


In [ ]:
## find the feature importances for each channels
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape

In [ ]:
channel1.shape

In [ ]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
## Drop the channels that have 0 importances
epoch.drop_channels(['LL13','RD6'])

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

In [ ]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

In [ ]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape

In [ ]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
##Drop channels that have 0 importances
epoch.drop_channels(['RR13','LD6','LA5','LL2'])

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

In [ ]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

In [ ]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape

In [ ]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
## Drop channels that have 0 importances
epoch.drop_channels(['R13','RA5','LD3','RE1'])

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

In [ ]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

In [ ]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape

In [ ]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
## Drop channels that have 0 importances
epoch.drop_channels(['LB6','Z5','Z4'])

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

In [ ]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

In [ ]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape

In [ ]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
## Drop channels that have 0 importances
epoch.drop_channels(['Z2','Z6'])

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

In [ ]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

In [ ]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape

In [ ]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
## Drop channels that have 0 importances
epoch.drop_channels(['RA3'])

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

In [ ]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

In [ ]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape

In [ ]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
## Drop channels that have 0 importances
epoch.drop_channels(['LB5','RR10'])

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

In [ ]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

In [ ]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
print(channel1.shape)
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
## Drop channels that have 0 importances
epoch.drop_channels(['L11','LC3'])

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

In [ ]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
print(channel1.shape)
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
## Drop channels that have 0 importances
epoch.drop_channels(['RC5','R2'])

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
print(channel1.shape)
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
## Drop channels that have 0 importances
epoch.drop_channels(['RB1','R4','Z14','RC2','RD7','LA4'])

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
print(channel1.shape)
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

In [ ]:
## Drop channels that have 0 importances
epoch.drop_channels(['L8','LL7','L1','Z12','L12','Z7','Nasium'])

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
print(channel1.shape)
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp